In [1]:
print('Hello World')

Hello World


In [ ]:
#~source: https: //en.wikipedia.org/wiki/Gradient_descent
# code source: https://en.wikipedia.org/w/index.php?title=Gradient_descent&oldid=966271567

# 初始值
next_x = 6# We start the search at x = 6
# 步长系数（学习率）
gamma = 0.01# Step size multiplier
# 提前停止（系数变化小于此值时停止）
precision = 0.00001# Desired precision of result
# 最大迭代次数
max_iters = 10000# Maximum number of iterations

# Derivative
#function
#求导
def df(x):
  return 4 * x ** 3 - 9 * x ** 2

# 迭代
for i in range(max_iters):
    current_x = next_x
    
    #梯度下降
    next_x = current_x - gamma * df(current_x)
    print(i, next_x, df(current_x))

    # 提前停止的判定，计算系数变化了多少，小于阈值后提前停止
    step = next_x - current_x
    if abs(step) <= precision:
        break

print("Minimum at ", next_x)

# The output for the above will be something like 
# "Minimum at 2.2499646074278457"